| 단계             | 핵심 로직                                                                      | 주요 라이브러리                                                          |
| -------------- | ---------------------------------------------------------------------------- | ----------------------------------------------------------------- |
| 📥 **로드**      | `pd.read_json()`으로 한 줄 JSON 배열을 DataFrame화                                   | pandas                                                            |
| 🧹 **전처리**     | `str.split('[SEP]')`로 **제목/본문 분리**, 특수문자·광고 제거,<br>512 토큰 관리 *(Truncation 또는 Summary)* | re, pandas                                                        |
| ✂️ **요약 (선택)** | 본문이 길면 **KoBART summarizer**로 3–5문장 요약                                     | transformers (`gogamza/kobart-summarization`)  |
| 🏷 **감정 분류**   | `snunlp/KR-FinBERT-SC` → Softmax → *pos/neu/neg* 확률                        | transformers, **torch** ([Hugging Face][2])                       |
| 📊 **집계**      | `groupby('stock_name').agg()`로 **종목별 평균·비율** 계산 → 일일 Sentiment Index       | pandas                                                            |
| 📤 **저장/연계**   | 기사단위 결과 `news_2025-08-05_sent_scored.jsonl`,<br>집계 결과 `daily_sentiment_20250805.csv` | —                                                                 |

[2]: https://huggingface.co/snunlp/KR-FinBERT-SC "snunlp/KR-FinBERT-SC on Hugging Face"


In [ ]:
# sentiment_pipeline.py
import re, time, json, torch, pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, pipeline,
    BartForConditionalGeneration
)

# ─────────────────────────────────────────────────────────────
# 0) 환경 설정
# ─────────────────────────────────────────────────────────────
DEVICE = "mps" 
# if torch.cuda.is_available() else "cpu"
# torch.set_num_threads(torch.get_num_threads())           # CPU 멀티스레드

tqdm.pandas(desc="Progress")                             # pandas + tqdm 결합
start_time = time.perf_counter()

def log(msg: str):
    elapsed = time.perf_counter() - start_time
    print(f"[{elapsed:6.1f}s] {msg}")

log(f"실행 디바이스: {DEVICE}")

# ─────────────────────────────────────────────────────────────
# 1) 데이터 로드
# ─────────────────────────────────────────────────────────────
INPUT_FILE = Path("/Users/yujimin/KB AI CHALLENGE/project/llm_input/news_2025-08-05.json")
df = pd.read_json(INPUT_FILE)
log(f"뉴스 로드 완료: {len(df):,}건")

# ─────────────────────────────────────────────────────────────
# 2) 제목 · 본문 분리
# ─────────────────────────────────────────────────────────────
def split_text(txt: str):
    title, *body = re.split(r'\s*\[SEP\]\s*', txt, maxsplit=1)
    return pd.Series([title.strip(), body[0].strip() if body else ""])

df[["title", "body"]] = df["text"].apply(split_text)
log("제목/본문 분리 완료")

# ─────────────────────────────────────────────────────────────
# 3) (옵션) KoBART 요약
# ─────────────────────────────────────────────────────────────
USE_SUMMARY = True
if USE_SUMMARY:
    log("KoBART 요약 모델 로딩 중…")
    SUM_MODEL_NAME = "gogamza/kobart-summarization"
    sum_tok = AutoTokenizer.from_pretrained(SUM_MODEL_NAME)
    sum_mod = BartForConditionalGeneration.from_pretrained(SUM_MODEL_NAME).to(DEVICE)

    @torch.inference_mode()
    def summarize(txt: str) -> str:
        inputs = sum_tok(txt, return_tensors="pt", max_length=1024,
                         truncation=True).to(DEVICE)
        inputs.pop("token_type_ids", None)   # BART는 필요 없음
        summary_ids = sum_mod.generate(**inputs,
                                       max_length=128, min_length=40,
                                       do_sample=False)
        return sum_tok.decode(summary_ids[0], skip_special_tokens=True).strip()

    log("본문 요약 시작…")
    df["body"] = df["body"].progress_apply(summarize)
    log("본문 요약 완료")

# ─────────────────────────────────────────────────────────────
# 4) KR-FinBERT 감정 분석
# ─────────────────────────────────────────────────────────────
log("FinBERT 모델 로딩 중…")
FINBERT_NAME = "snunlp/KR-FinBERT-SC"
tok = AutoTokenizer.from_pretrained(FINBERT_NAME)
mod = AutoModelForSequenceClassification.from_pretrained(FINBERT_NAME).to(DEVICE)

@torch.inference_mode()
def classify(title: str, body: str):
    enc = tok(title, body, truncation=True,
              max_length=512, return_tensors="pt").to(DEVICE)
    probs = mod(**enc).logits.softmax(-1)[0].cpu().tolist()   # [neg, neu, pos]
    return probs

def score_row(row):
    neg, neu, pos = classify(row.title, row.body)
    return pd.Series({"neg": neg, "neu": neu, "pos": pos})

log("감정 점수 산출 중…")
df[["neg", "neu", "pos"]] = df.progress_apply(score_row, axis=1)
log("감정 분석 완료")

# ─────────────────────────────────────────────────────────────
# 5) 종목별 집계
# ─────────────────────────────────────────────────────────────
agg = (df.groupby("stock_name")
         .agg(article_cnt=("pos", "size"),
              pos_mean=("pos", "mean"),
              neg_mean=("neg", "mean"),
              neu_mean=("neu", "mean"))
         .reset_index())
log("집계 완료")

# ─────────────────────────────────────────────────────────────
# 6) 결과 저장
# ─────────────────────────────────────────────────────────────
# df_out  = Path("news_2025-08-05_sent_scored.jsonl")
df_out  = Path("news_2025-08-05_sent_scored.csv")
agg_out = Path("daily_sentiment_20250805.csv")

# df.to_json(df_out, orient="records", lines=True, force_ascii=False)
df.to_csv(df_out, index=False)
agg.to_csv(agg_out, index=False)
log(f"파일 저장 완료 → {df_out} / {agg_out}")


[   0.0s] 실행 디바이스: mps
[   0.1s] 뉴스 로드 완료: 728건
[   0.1s] 제목/본문 분리 완료
[   0.1s] KoBART 요약 모델 로딩 중…


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


[   3.7s] 본문 요약 시작…


Progress:   0%|          | 0/728 [00:00<?, ?it/s]

[1607.7s] 본문 요약 완료
[1607.7s] FinBERT 모델 로딩 중…
[1610.5s] 감정 점수 산출 중…


Progress:   0%|          | 0/728 [00:00<?, ?it/s]

[1651.0s] 감정 분석 완료
[1651.0s] 집계 완료
[1651.1s] 파일 저장 완료 → news_2025-08-05_sent_scored.jsonl / daily_sentiment_20250805.csv


In [12]:
df_out  = Path("news_2025-08-05_sent_scored.csv")
df.to_csv(df_out, index=False)